In [ ]:
apikey= 'SEDcFR325HFEwsxuz'
apisecretkey= 'SEDcFR325HFEwsxuzSEDcFR325HFEwsxuzSEDcFR325HFEwsxuz'
accesstoken='126SEDcFR325HFEwsxuzbLHXKSEDcFR325HFEwsxuz6m56xlvGSEDcFR325HFEwsxuz'
accesstokensecret='JS1SEDcFR325HFEwsxuzTCsX6g8JpLzFA5SEDcFR325HFEwsxuzD'

In [ ]:
import os
import tweepy as tw

auth = tw.OAuthHandler(apikey, apisecretkey)
auth.set_access_token(accesstoken, accesstokensecret)

api = tw.API(auth,wait_on_rate_limit=True)

In [ ]:
import pandas as pd
def extract_timeline_as_df(timeline_list):
    columns = set()
    allowed_types = [str, int]
    tweets_data = []
    for status in timeline_list:
        status_dict = dict(vars(status))
        keys = status_dict.keys()
        single_tweet_data = {"user": status.user.screen_name, "author": status.author.screen_name}
        for k in keys:
            try:
                v_type = type(status_dict[k])
            except:
                v_type = None
            if v_type != None:
                if v_type in allowed_types:
                    single_tweet_data[k] = status_dict[k]
                    columns.add(k)
        tweets_data.append(single_tweet_data)


    header_cols = list(columns)
    header_cols.append("user")
    header_cols.append('author')
    df = pd.DataFrame(tweets_data, columns=header_cols)
    return df

In [ ]:
#1: What's on MY timeline
my_timeline = api.home_timeline()
df2 = extract_timeline_as_df(my_timeline)
df2.head(3)

In [ ]:
#2: What's on different user's timeline
user = api.get_user("JioMart")#JioMart
user_timeline = user.timeline()
df3 = extract_timeline_as_df(user_timeline)
df3.head(2)

In [ ]:
#3 Find all followers and friends count
user = api.get_user("JioMart")
print(user.followers_count, user.friends_count)

In [ ]:
#4 #following
user_friends = user.friends()
for friend in user_friends:
  print(friend.screen_name)

In [ ]:
#5: My "Tweets & replies"
import tweepy
for i, status in enumerate(tweepy.Cursor(api.home_timeline, count=5).items(4)):
    print(i, status.text)

In [ ]:
#6_1 - Retrieve Tweets
from textblob import TextBlob
public_tweets = api.search('JioMart')

for tweet in public_tweets:
    print(tweet.text)
    
    #Step 4 Perform Sentiment Analysis on Tweets
    analysis = TextBlob(tweet.text)
    print(analysis.sentiment)
    print("")

In [ ]:
#6_1: Track hashtag from any date
!pip install -q snscrape
import os
import pandas as pd
from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/My Drive/Colab Notebooks/POC_Projects/'

from datetime import date, timedelta
today = date.today()
end_date = today

search_term = 'JioMart'
from_date = date.today() - timedelta(days=10)# '2022-08-28'

os.system(f"snscrape --since {from_date} twitter-search '{search_term} until:{end_date}' > Date_Range_tweets.txt") #will take more than 10 minutes
if os.stat("Date_Range_tweets.txt").st_size == 0:
  counter = 0
else:
  df = pd.read_csv('Date_Range_tweets.txt', names=['link'])
  counter = df.size

print('Number Of Tweets : '+ str(counter))

In [ ]:
#6_1_Extracting Exact Tweeets -- may take 4 to 5 mins
max_results = 5383 #add the number as per ABOVE 'Number Of Tweets'
extracted_tweets = "snscrape --format '{content!r}'"+ f" --max-results {max_results} --since {from_date} twitter-search '{search_term} until:{end_date}' > Date_Range_tweets.txt"

# extracted_tweets = (re.sub('@[a-z,_,A-Z,0-9,:]+ ', '', i) for i in extracted_tweets)

os.system(extracted_tweets)
if os.stat("Date_Range_tweets.txt").st_size == 0:
  print('No Tweets found')
else:
  df = pd.read_csv('Date_Range_tweets.txt', names=['content'])
  for row in df['content'].iteritems():
    print(row)

In [ ]:
#6: Track hashtag from any date
search_words = ["#JioMart"]
date_since = "2022-11-01"

tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items() # limit the count using e.g. .items(5)

tweets

tweet_details = [[tweet.geo, tweet.text,tweet.user.screen_name, tweet.user.location,tweet.created_at,[e['text'] for e in tweet._json['entities']['hashtags']], tweet.user.followers_count] for tweet in tweets]
# tweet_details

import pandas as pd
tweet_df = pd.DataFrame(data=tweet_details, columns=['geo','text','user', "location","Timestamp","All_hashtags","Followers_Count"])

pd.set_option('max_colwidth', 80)
tweet_df.head(5)

In [ ]:
#7: User's count who tweeted
tweet_df.user.value_counts()

In [ ]:
#8: Tweeted from location
tweet_df.location.value_counts()

In [ ]:
#9: Save csv file after data cleaning
import re
def clean_tweets(text):
    text = re.sub("RT @[\w]*:","",text)
    text = re.sub("@[\w]*","",text)
    text = re.sub("https?://[A-Za-z0-9./]*","",text)
    text = re.sub("\n","",text)


    text = re.sub("\$\w*","",text)
    text = re.sub("^RT[\s]+","",text)
    text = re.sub("https?:\/\/.*[\r\n]*","",text)
    text = re.sub("#","",text)

    return text

tweet_df['text']=tweet_df['text'].apply(lambda x: clean_tweets(x))

# tweet_df.head(20)
tweet_df.to_csv('Hashed_tweets.csv') #check file in right side folder
# !ls

In [ ]:
#10 SENTIMENT ANALYSIS
import spacy
nlp = spacy.load('en_core_web_sm')

tweet_df['text'].apply(lambda x: [print("\tText : {}, Entity : {}".format(ent.text, ent.label_)) if (not ent.text.startswith('#')) else ""  for ent in nlp(x).ents])


In [ ]:

tweet_df['entities']=tweet_df['text'].apply(lambda x: [(ent.text, ent.label_) if (not ent.text.startswith('#')) else "" for ent in nlp(x).ents])
print(tweet_df.head(5))

In [ ]:
#7: NER
import nltk
nltk.download('punkt')

from nltk import ne_chunk
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import word_tokenize
def ner_tweets(text):
  ne_token =word_tokenize(text)
  ne_tags = nltk.pos_tag(ne_token)

  ne_ner = ne_chunk(ne_tags)
  return ne_ner

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()

tweet_df['sentiment']=tweet_df['text'].apply(lambda x: sid.polarity_scores(x))
tweet_df['NER_1']=tweet_df['text'].apply(lambda x: [(ent.text,ent.start_char, ent.end_char, ent.label_) if (not ent.text.startswith('#')) else "" for ent in nlp(x).ents])
tweet_df['NER_2']=tweet_df['text'].apply(lambda x: ner_tweets(x))


tweet_df.to_csv('FINAL.csv')
tweet_df.head(10) #sentiment is added

In [ ]:
#11: Export csv file as per inputted hashtag
import json
import csv
import tweepy
def search_for_hashtags(apikey, apisecretkey, accesstoken, accesstokensecret, hashtag_phrase):
    #create authentication for accessing Twitter
    auth = tweepy.OAuthHandler(apikey, apisecretkey)
    auth.set_access_token(accesstoken, accesstokensecret)

    #initialize Tweepy API
    api = tweepy.API(auth)
    
    #get the name of the spreadsheet we will write to
    fname = ''.join(re.findall(r"#(\w+)", hashtag_phrase))

    #open the spreadsheet we will write to
    with open('%s 2_tweets.csv' % (fname), 'w') as file:

      w = csv.writer(file)

      #write header row to spreadsheet
      w.writerow(['timestamp', 'tweet_text', 'username', 'all_hashtags', 'followers_count'])

      #for each tweet matching our hashtags, write relevant info to the spreadsheet
      for tweet in tweepy.Cursor(api.search, q=hashtag_phrase+' -filter:retweets', lang="en", tweet_mode='extended').items(10):
        w.writerow([tweet.created_at, tweet.full_text.replace('\n',' ').encode('utf-8'), tweet.user.screen_name.encode('utf-8'), [e['text'] for e in tweet._json['entities']['hashtags']], tweet.user.followers_count])
            


# hashtag_phrase = input('Hashtag Phrase: ') ##FIFAWorldCup
hashtag_phrase = 'FIFAWorldCup' ##FIFAWorldCup
if __name__ == '__main__':
  search_for_hashtags(apikey, apisecretkey, accesstoken, accesstokensecret, hashtag_phrase)

In [ ]:
#12: Reply any tweet
# og_tweet = api.get_status("1584907651753746432")
# # print(og_tweet.user.screen_name, og_tweet.id)
# my_reply = api.update_status(f"@{og_tweet.user.screen_name} Wow this cool!", og_tweet.id)
#print(my_reply.id, my_reply.user.screen_name)

In [ ]:
#13: Word Cloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt 
text = " ".join(review for review in tweet_df.text)
# Generate a word cloud image
wordcloud = WordCloud(max_font_size=50, max_words=1000,background_color="white",contour_width=3, contour_color='firebrick').generate(text)

# Display the generated image:
# the matplotlib way:
plt.figure(figsize=[20,10])
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

# store to file
plt.savefig("WC.png", format="png")